In [1]:
######################################################################
###############   IMPORTING THE NECESSARY PACKAGES  ##################
######################################################################

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [4]:
######################################################################
##################   DEVICE CONFIGURATION ############################
######################################################################

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [5]:
######################################################################
##################  HYPTERPARAMETER DEFINITION #######################
######################################################################

batch_size = 100
num_epochs = 80
learning_rate = 0.001

In [6]:
######################################################################
##################  PREPROCESSING OF IMAGE ###########################
######################################################################

transform = transforms.Compose([transforms.Pad(4), transforms.RandomHorizontalFlip(),
                               transforms.RandomCrop(32), transforms.ToTensor()])

In [13]:
######################################################################
#####################  PREPARING THE TRAINING DATA ###################
######################################################################

train_dataset = torchvision.datasets.CIFAR10(root = './data', train = True,
                                             transform = transform, 
                                             download = True)
test_dataset = torchvision.datasets.CIFAR10(root = './data', train = False,
                                            transform = transforms.ToTensor()
                                           )

trainLoader = torch.utils.data.DataLoader(dataset = train_dataset, 
                                                batch_size = batch_size, 
                                                shuffle = True)
testLoader = torch.utils.data.DataLoader(dataset = test_dataset,
                                               batch_size = batch_size,
                                              shuffle = False)

Files already downloaded and verified


In [ ]:
######################################################################
##################### DEFINING THE MODEL CLASSES #####################
######################################################################

def conv3x3(self, in_channels, out_channels, stride = 1):
    return(nn.Conv2d(in_channels, out_channels, kernel_size = 3, 
                    stride = stride, padding = 1, bias = False))

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super(ResidualBlock, self).__init__()
        self.convv1 = (conv3x3(in_channels, out_channels, stride = 1))
    
